# Creating a Sampled Dataset

**Learning Objectives**

1. Setup up the environment
1. Sample the natality dataset to create train, eval, test sets
1. Preprocess the data in Pandas dataframe


## Introduction 
In this notebook, we'll read data from BigQuery into our notebook to preprocess the data within a Pandas dataframe for a small, repeatable sample.

We will set up the environment, sample the natality dataset to create train, eval, test splits, and preprocess the data in a Pandas dataframe.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/tree/master/courses/machine_learning/deepdive2/end_to_end_ml/solutions/sample_babyweight.ipynb).

## Set up environment variables and load necessary libraries

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
!pip install --user google-cloud-bigquery==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 KB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 2.2.1 requires google-resumable-media>=2.3.2, but you have google-resumable-media 0.5.1 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.41.0 which is incompatible.


**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Import necessary libraries.

In [1]:
from google.cloud import bigquery
import pandas as pd

**Lab Task #1:** Set up environment variables so that we can use them throughout the notebook

In [2]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-02-7b86a7f2cc51


In [3]:
PROJECT = "qwiklabs-gcp-02-7b86a7f2cc51"  # Replace with your PROJECT

## Create ML datasets by sampling using BigQuery

We'll begin by sampling the BigQuery data to create smaller datasets. Let's create a BigQuery client that we'll use throughout the lab.

In [4]:
bq = bigquery.Client(project = PROJECT)

We need to figure out the right way to divide our hash values to get our desired splits. To do that we need to define some values to hash within the module. Feel free to play around with these values to get the perfect combination.

In [5]:
modulo_divisor = 100
train_percent = 80.0
eval_percent = 10.0

train_buckets = int(modulo_divisor * train_percent / 100.0)
eval_buckets = int(modulo_divisor * eval_percent / 100.0)

We can make a series of queries to check if our bucketing values result in the correct sizes of each of our dataset splits and then adjust accordingly. Therefore, to make our code more compact and reusable, let's define a function to return the head of a dataframe produced from our queries up to a certain number of rows.

In [6]:
def display_dataframe_head_from_query(query, count=10):
    """Displays count rows from dataframe head from query.
    
    Args:
        query: str, query to be run on BigQuery, results stored in dataframe.
        count: int, number of results from head of dataframe to display.
    Returns:
        Dataframe head with count number of results.
    """
    df = bq.query(
        query + " LIMIT {limit}".format(
            limit=count)).to_dataframe()

    return df.head(count)

For our first query, we're going to use the original query above to get our label, features, and columns to combine into our hash which we will use to perform our repeatable splitting. There are only a limited number of years, months, days, and states in the dataset. Let's see what the hash values are. We will need to include all of these extra columns to hash on to get a fairly uniform spread of the data. Feel free to try less or more in the hash and see how it changes your results.

In [7]:
# Get label, features, and columns to hash and split into buckets
hash_cols_fixed_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
"""

display_dataframe_head_from_query(hash_cols_fixed_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,year,month,date,state,mother_birth_state
0,6.686620,True,18,1,43,2001,5,3,FL,Foreign
1,9.360828,True,32,1,41,2001,8,2,KS,KS
2,8.437091,False,30,1,39,2001,9,2,KY,KY
3,6.124442,False,24,1,40,2002,10,2,AL,AL
4,7.125340,False,26,1,41,2002,5,5,AZ,AZ
5,6.944561,False,31,1,40,2001,10,3,CA,IL
6,6.999677,True,23,1,40,2001,8,2,MS,MS
7,9.374055,True,34,1,40,2001,2,6,NY,NJ
8,8.373157,True,33,1,40,2001,11,3,OR,CA
9,7.187070,False,23,1,40,2001,7,2,TX,TX


Using `COALESCE` would provide the same result as the nested `CASE WHEN`. This is preferable when all we want is the first non-null instance. To be precise the `CASE WHEN` would become `COALESCE(wday, day, 0) AS date`. You can read more about it [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions).

Next query will combine our hash columns and will leave us just with our label, features, and our hash values.

In [8]:
data_query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    ({CTE_hash_cols_fixed})
""".format(CTE_hash_cols_fixed=hash_cols_fixed_query)

display_dataframe_head_from_query(data_query)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,8.818490,False,17,1,42,6174834893904193287
1,8.141671,False,29,1,38,-4679450203487894236
2,5.948072,True,38,1,38,5830021534043063509
3,8.838332,True,27,1,39,-6484105101089659230
4,9.259415,True,28,1,38,-7561476152214523685
5,7.125340,True,34,1,39,4100584960946904090
6,7.063611,False,18,1,40,870533685982839311
7,7.500126,True,31,1,44,7999589519823312102
8,6.250105,False,23,1,41,-4634431482337591685
9,3.999185,False,30,1,32,-6873880094893634931


The next query is going to find the counts of each of the unique 657484 `hash_values`. This will be our first step at making actual hash buckets for our split via the `GROUP BY`.

In [9]:
# Get the counts of each of the unique hash of our splitting column
first_bucketing_query = """
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    ({CTE_data})
GROUP BY
    hash_values
""".format(CTE_data=data_query)

display_dataframe_head_from_query(first_bucketing_query)

,hash_values,num_records
0,-2152741494508244169,226
1,8834092610108129167,331
2,4334988609252370242,2899
3,3879253354687497353,392
4,-7937880894327351264,4
5,-341510792376592286,2
6,5819299517989765743,324
7,869714065522591735,9
8,-5368317049176962418,1376
9,6162333716261329094,1125


The query below performs a second layer of bucketing where now for each of these bucket indices we count the number of records.

In [26]:
# Get the number of records in each of the hash buckets
second_bucketing_query = """
SELECT
    ABS(MOD(hash_values, {modulo_divisor})) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    ({CTE_first_bucketing})
GROUP BY
    ABS(MOD(hash_values, {modulo_divisor}))
""".format(
    CTE_first_bucketing=first_bucketing_query, modulo_divisor=modulo_divisor)
print(second_bucketing_query)
display_dataframe_head_from_query(second_bucketing_query)


SELECT
    ABS(MOD(hash_values, 100)) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    (
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    (
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    (
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    A

,bucket_index,num_records
0,60,355099
1,92,336735
2,19,384793
3,63,355283
4,66,402627
5,76,354090
6,86,274489
7,18,408053
8,30,333513
9,81,233538


The number of records is hard for us to easily understand the split, so we will normalize the count into percentage of the data in each of the hash buckets in the next query.

In [27]:
# Calculate the overall percentages
percentages_query = """
SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
        SUM(num_records)
    FROM
        ({CTE_second_bucketing})) AS percent_records
FROM
    ({CTE_second_bucketing})
""".format(CTE_second_bucketing=second_bucketing_query)

display_dataframe_head_from_query(percentages_query)

,bucket_index,num_records,percent_records
0,48,370308,0.011218
1,17,222562,0.006742
2,98,374697,0.011351
3,97,480790,0.014564
4,91,333267,0.010096
5,8,370758,0.011231
6,4,398118,0.012060
7,62,426834,0.012930
8,51,180001,0.005453
9,57,453019,0.013723


We'll now select the range of buckets to be used in training.

In [29]:
# Choose hash buckets for training and pull in their statistics
train_query = """
SELECT
    *,
    "train" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= 0
    AND bucket_index < {train_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets)

print(train_query)

display_dataframe_head_from_query(train_query)


SELECT
    *,
    "train" AS dataset_name
FROM
    (
SELECT
    bucket_index,
    num_records,
    CAST(num_records AS FLOAT64) / (
    SELECT
        SUM(num_records)
    FROM
        (
SELECT
    ABS(MOD(hash_values, 100)) AS bucket_index,
    SUM(num_records) AS num_records
FROM
    (
SELECT
    hash_values,
    COUNT(*) AS num_records
FROM
    (
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    ) AS hash_values
FROM
    (
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    year,
    month,
    CASE
        WHEN day IS NULL THEN
            CASE
                WHEN wday IS NULL THEN 0
                ELSE wday
            END
        ELSE day
    END AS date,
    IF

,bucket_index,num_records,percent_records,dataset_name
0,28,449682,0.013622,train
1,12,412875,0.012507,train
2,50,184434,0.005587,train
3,5,449280,0.013610,train
4,26,492824,0.014929,train
5,56,226752,0.006869,train
6,40,333712,0.010109,train
7,52,204972,0.006209,train
8,33,410226,0.012427,train
9,79,403701,0.012229,train


We'll do the same by selecting the range of buckets to be used evaluation.

In [30]:
# Choose hash buckets for validation and pull in their statistics
eval_query = """
SELECT
    *,
    "eval" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {train_buckets}
    AND bucket_index < {cum_eval_buckets}
""".format(
    CTE_percentages=percentages_query,
    train_buckets=train_buckets,
    cum_eval_buckets=train_buckets + eval_buckets)

display_dataframe_head_from_query(eval_query)

,bucket_index,num_records,percent_records,dataset_name
0,82,468179,0.014182,eval
1,80,312489,0.009466,eval
2,85,368045,0.011149,eval
3,87,523881,0.015870,eval
4,86,274489,0.008315,eval
5,81,233538,0.007074,eval
6,83,411258,0.012458,eval
7,89,256482,0.007770,eval
8,88,423809,0.012838,eval
9,84,341155,0.010334,eval


Lastly, we'll select the hash buckets to be used for the test split.

In [31]:
# Choose hash buckets for testing and pull in their statistics
test_query = """
SELECT
    *,
    "test" AS dataset_name
FROM
    ({CTE_percentages})
WHERE
    bucket_index >= {cum_eval_buckets}
    AND bucket_index < {modulo_divisor}
""".format(
    CTE_percentages=percentages_query,
    cum_eval_buckets=train_buckets + eval_buckets,
    modulo_divisor=modulo_divisor)

display_dataframe_head_from_query(test_query)

,bucket_index,num_records,percent_records,dataset_name
0,98,374697,0.011351,test
1,97,480790,0.014564,test
2,91,333267,0.010096,test
3,94,431001,0.013056,test
4,99,223334,0.006765,test
5,96,529357,0.016036,test
6,95,313544,0.009498,test
7,92,336735,0.010201,test
8,90,286465,0.008678,test
9,93,215710,0.006534,test


In the below query, we'll `UNION ALL` all of the datasets together so that all three sets of hash buckets will be within one table. We added `dataset_id` so that we can sort on it in the query after.

In [32]:
# Union the training, validation, and testing dataset statistics
union_query = """
SELECT
    0 AS dataset_id,
    *
FROM
    ({CTE_train})
UNION ALL
SELECT
    1 AS dataset_id,
    *
FROM
    ({CTE_eval})
UNION ALL
SELECT
    2 AS dataset_id,
    *
FROM
    ({CTE_test})
""".format(CTE_train=train_query, CTE_eval=eval_query, CTE_test=test_query)

display_dataframe_head_from_query(union_query)

,dataset_id,bucket_index,num_records,percent_records,dataset_name
0,1,80,312489,0.009466,eval
1,2,94,431001,0.013056,test
2,0,31,220366,0.006675,train
3,0,49,468121,0.014181,train
4,0,42,405587,0.012286,train
5,0,27,355234,0.010761,train
6,0,34,379000,0.011481,train
7,0,37,295153,0.008941,train
8,0,78,326758,0.009898,train
9,0,7,270933,0.008207,train


Lastly, we'll show the final split between train, eval, and test sets. We can see both the number of records and percent of the total data. It is really close to that we were hoping to get.

In [33]:
# Show final splitting and associated statistics
split_query = """
SELECT
    dataset_id,
    dataset_name,
    SUM(num_records) AS num_records,
    SUM(percent_records) AS percent_records
FROM
    ({CTE_union})
GROUP BY
    dataset_id,
    dataset_name
ORDER BY
    dataset_id
""".format(CTE_union=union_query)

display_dataframe_head_from_query(split_query)

,dataset_id,dataset_name,num_records,percent_records
0,0,train,25873134,0.783765
1,1,eval,3613325,0.109457
2,2,test,3524900,0.106778


Now that we know that our splitting values produce a good global splitting on our data, here's a way to get a well-distributed portion of the data in such a way that the train, eval, test sets do not overlap and takes a subsample of our global splits.

**Lab Task #2:** Sample the natality dataset

In [34]:
# every_n allows us to subsample from each of the hash values
# This helps us get approximately the record counts we want
every_n = 1000

splitting_string = "ABS(MOD(hash_values, {0} * {1}))".format(every_n, modulo_divisor)

def create_data_split_sample_df(query_string, splitting_string, lo, up):
    """Creates a dataframe with a sample of a data split.

    Args:
        query_string: str, query to run to generate splits.
        splitting_string: str, modulo string to split by.
        lo: float, lower bound for bucket filtering for split.
        up: float, upper bound for bucket filtering for split.
    Returns:
        Dataframe containing data split sample.
    """
    query = "SELECT * FROM ({0}) WHERE {1} >= {2} and {1} < {3}".format(
        query_string, splitting_string, int(lo), int(up))

    df = bq.query(query).to_dataframe()

    return df

train_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=0, up=train_percent)

eval_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent, up=train_percent + eval_percent)

test_df = create_data_split_sample_df(
    data_query, splitting_string,
    lo=train_percent + eval_percent, up=modulo_divisor)

print("There are {} examples in the train dataset.".format(len(train_df)))
print("There are {} examples in the validation dataset.".format(len(eval_df)))
print("There are {} examples in the test dataset.".format(len(test_df)))

There are 7733 examples in the train dataset.
There are 1037 examples in the validation dataset.
There are 561 examples in the test dataset.


## Preprocess data using Pandas

We'll perform a few preprocessing steps to the data in our dataset. Let's add extra rows to simulate the lack of ultrasound. That is we'll duplicate some rows and make the `is_male` field be `Unknown`. Also, if there is more than child we'll change the `plurality` to `Multiple(2+)`. While we're at it, we'll also change the plurality column to be a string. We'll perform these operations below. 

Let's start by examining the training dataset as is.

In [35]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,6.977631,False,38,1,38,-6784884401981100070
1,7.561856,False,34,1,40,4896699230184800022
2,6.161920,True,33,1,36,-7200461189606400005
3,6.406633,True,33,1,41,-6784884401981100070
4,7.874912,True,29,1,39,4896699230184800022


Also, notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [36]:
train_df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hash_values
count,7733.000000,7733.000000,7733.000000,7733.000000,7.733000e+03
mean,7.264415,28.213371,1.035691,38.691064,-2.984870e+17
std,1.303220,6.134232,0.201568,2.531921,5.590715e+18
min,0.562179,13.000000,1.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,1.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,1.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,1.000000,40.000000,4.896699e+18
max,11.563246,48.000000,4.000000,47.000000,9.203641e+18


It is always crucial to clean raw data before using in machine learning, so we have a preprocessing step. We'll define a `preprocess` function below. Note that the mother's age is an input to our model so users will have to provide the mother's age; otherwise, our service won't work. The features we use for our model were chosen because they are such good predictors and because they are easy enough to collect.

**Lab Task #3:** Preprocess the data in Pandas dataframe

In [37]:
def preprocess(df):
    """ Preprocess pandas dataframe for augmented babyweight data.
    
    Args:
        df: Dataframe containing raw babyweight data.
    Returns:
        Pandas dataframe containing preprocessed raw babyweight data as well
            as simulated no ultrasound data masking some of the original data.
    """
    # Clean up raw data
    # Filter out what we don"t want to use for training
    df = df[df.weight_pounds > 0]
    df = df[df.mother_age > 0]
    df = df[df.gestation_weeks > 0]
    df = df[df.plurality > 0]

    # Modify plurality field to be a string
    twins_etc = dict(zip([1,2,3,4,5],
                   ["Single(1)",
                    "Twins(2)",
                    "Triplets(3)",
                    "Quadruplets(4)",
                    "Quintuplets(5)"]))
    df["plurality"].replace(twins_etc, inplace=True)

    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound = df.copy(deep=True)

    # Modify is_male
    no_ultrasound["is_male"] = "Unknown"
    
    # Modify plurality
    condition = no_ultrasound["plurality"] != "Single(1)"
    no_ultrasound.loc[condition, "plurality"] = "Multiple(2+)"

    # Concatenate both datasets together and shuffle
    return pd.concat(
        [df, no_ultrasound]).sample(frac=1).reset_index(drop=True)

Let's process the train, eval, test set and see a small sample of the training data after our preprocessing:

In [38]:
train_df = preprocess(train_df)
eval_df = preprocess(eval_df)
test_df = preprocess(test_df)

In [39]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
0,5.952481,True,33,Twins(2),37,-6784884401981100070
1,5.187477,Unknown,33,Single(1),36,4896699230184800022
2,6.305221,Unknown,24,Single(1),40,-7615106494593500018
3,9.376260,True,36,Single(1),39,6567453535821100019
4,7.687519,Unknown,27,Single(1),40,-3721975748227700022


In [40]:
train_df.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hash_values
15461,8.311427,Unknown,29,Single(1),40,3620607574719600019
15462,9.124933,Unknown,29,Single(1),38,-4614303140002600076
15463,8.501025,Unknown,27,Single(1),38,-5582603752607400050
15464,6.188376,False,32,Single(1),40,4896699230184800022
15465,7.749249,Unknown,34,Single(1),42,-6784884401981100070


Let's look again at a summary of the dataset. Note that we only see numeric columns, so `plurality` does not show up.

In [41]:
train_df.describe()

,weight_pounds,mother_age,gestation_weeks,hash_values
count,15466.000000,15466.000000,15466.000000,1.546600e+04
mean,7.264415,28.213371,38.691064,-2.984870e+17
std,1.303178,6.134034,2.531839,5.590534e+18
min,0.562179,13.000000,18.000000,-9.210618e+18
25%,6.624891,23.000000,38.000000,-6.781866e+18
50%,7.345803,28.000000,39.000000,5.057323e+17
75%,8.062305,33.000000,40.000000,4.896699e+18
max,11.563246,48.000000,47.000000,9.203641e+18


## Write to .csv files 

In the final versions, we want to read from files, not Pandas dataframes. So, we write the Pandas dataframes out as csv files. Using csv files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.

In [42]:
# Define columns
columns = ["weight_pounds",
           "is_male",
           "mother_age",
           "plurality",
           "gestation_weeks"]

# Write out CSV files
train_df.to_csv(
    path_or_buf="train.csv", columns=columns, header=False, index=False)
eval_df.to_csv(
    path_or_buf="eval.csv", columns=columns, header=False, index=False)
test_df.to_csv(
    path_or_buf="test.csv", columns=columns, header=False, index=False)

In [43]:
%%bash
wc -l *.csv

  2074 eval.csv
  1122 test.csv
 15466 train.csv
 18662 total


In [44]:
%%bash
head *.csv

==> eval.csv <==
6.93794738514,Unknown,32,Single(1),39
8.811876612139999,Unknown,40,Single(1),39
7.936641432,False,36,Single(1),38
5.43659938092,False,23,Single(1),37
7.62578964258,False,39,Single(1),38
6.87401332916,False,40,Single(1),40
7.31273323054,Unknown,30,Single(1),40
7.7492485093,Unknown,33,Single(1),40
8.1240343547,Unknown,22,Single(1),39
8.313631900019999,Unknown,35,Single(1),39

==> test.csv <==
5.74965579296,True,23,Single(1),35
6.75055446244,Unknown,22,Single(1),42
7.25100379718,Unknown,37,Single(1),39
8.3665428429,Unknown,16,Single(1),40
6.9996768185,Unknown,32,Single(1),39
6.3603362587,True,31,Single(1),39
7.87270737602,True,27,Single(1),39
5.81138522632,Unknown,37,Single(1),37
6.1883756943399995,True,32,Single(1),37
7.7492485093,True,30,Single(1),41

==> train.csv <==
5.952481074,True,33,Twins(2),37
5.18747702486,Unknown,33,Single(1),36
6.3052206932,Unknown,24,Single(1),40
9.37626000286,True,36,Single(1),39
7.68751907594,Unknown,27,Single(1),40
7.06140625186,Unknown,27

In [45]:
%%bash
tail *.csv

==> eval.csv <==
8.56275425608,True,37,Single(1),41
7.62358501996,True,30,Single(1),38
6.75055446244,Unknown,24,Single(1),39
7.5618555866,True,31,Single(1),39
7.62578964258,Unknown,42,Single(1),39
6.8122838958,True,25,Single(1),40
7.3744626639,Unknown,33,Single(1),40
7.81318256528,Unknown,22,Single(1),40
8.062304921339999,Unknown,30,Single(1),38
7.3744626639,Unknown,25,Single(1),39

==> test.csv <==
8.062304921339999,Unknown,25,Single(1),39
6.37576861704,Unknown,28,Single(1),39
8.24969784404,False,32,Single(1),37
7.83522879148,Unknown,23,Single(1),40
7.3744626639,True,19,Single(1),38
6.5697754076,True,32,Single(1),37
7.4317828520199996,False,30,Single(1),40
7.68751907594,Unknown,28,Single(1),38
7.87491199864,False,39,Single(1),38
7.25100379718,Unknown,41,Multiple(2+),40

==> train.csv <==
6.1244416383599996,Unknown,35,Single(1),38
5.43659938092,False,28,Single(1),36
7.68751907594,True,26,Single(1),44
8.000575487979999,False,35,Single(1),39
8.18796841068,True,25,Single(1),40
8.311427277

## Lab Summary: 
In this lab, we set up the environment, sampled the natality dataset to create train, eval, test splits, and preprocessed the data in a Pandas dataframe.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License